In [25]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
data = pd.read_csv('diabetes.csv')

data

In [4]:
x = data.iloc[:,0:-1].values
y_string = list(data.iloc[:,-1])

x

y_string

In [9]:
y_int = []
for string in y_string:
    if string == "positive":
        y_int.append(1)
    else:
        y_int.append(0)

y_int

In [12]:
y = np.array(y_int, dtype = 'float64')

In [13]:
sc = StandardScaler()
x = sc.fit_transform(x)

x

In [15]:
x = torch.tensor(x)
y = torch.tensor(y)

In [21]:
y.shape

torch.Size([768, 1])

In [20]:
y = y.unsqueeze(1)

In [22]:
class Dataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return len(self.x)
    

In [23]:
dataset = Dataset(x,y)

In [24]:
len(dataset)

768

In [26]:
train_loader = DataLoader(dataset= dataset, batch_size = 32, shuffle = True)

In [27]:
class Model(nn.Module):
    def __init__(self, input_features, output_features):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(input_features,5)
        self.fc2 = nn.Linear(5,4)
        self.fc3 = nn.Linear(4,3)
        self.fc4 = nn.Linear(3,1)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        out = self.tanh(out)
        out = self.fc3(out)
        out = self.tanh(out)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out
    

In [28]:
net = Model(7,1)

In [30]:
criterion = nn.BCELoss(reduction= 'mean')

In [31]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.1, momentum= 0.9)

In [32]:
epochs = 200
for epoch in range(epochs):
    for inputs,labels in train_loader:
        inputs = inputs.float()
        labels = labels.float()
        #forward prop
        outputs = net(inputs) #skrócona wersja zapisu net.forward(inputs)
        #loss Calc
        loss = criterion(outputs,labels)
        #clear gradient buffer - only in PyTorch, as standard, frameworks do it automatically
        optimizer.zero_grad()
        #backpropagation
        loss.backward()
        #update weights
        optimizer.step()
        
    #accuracy calc
    output = (outputs > 0.5).float()
    accuracy = (output == labels).float().mean()
    print("Epoch {}/ {}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,epochs,loss,accuracy))

Epoch 1/ 200, Loss: 0.680, Accuracy: 0.562
Epoch 2/ 200, Loss: 0.372, Accuracy: 0.875
Epoch 3/ 200, Loss: 0.594, Accuracy: 0.812
Epoch 4/ 200, Loss: 0.415, Accuracy: 0.719
Epoch 5/ 200, Loss: 0.503, Accuracy: 0.719
Epoch 6/ 200, Loss: 0.445, Accuracy: 0.750
Epoch 7/ 200, Loss: 0.605, Accuracy: 0.688
Epoch 8/ 200, Loss: 0.302, Accuracy: 0.906
Epoch 9/ 200, Loss: 0.471, Accuracy: 0.750
Epoch 10/ 200, Loss: 0.657, Accuracy: 0.656
Epoch 11/ 200, Loss: 0.596, Accuracy: 0.625
Epoch 12/ 200, Loss: 0.377, Accuracy: 0.812
Epoch 13/ 200, Loss: 0.434, Accuracy: 0.750
Epoch 14/ 200, Loss: 0.424, Accuracy: 0.844
Epoch 15/ 200, Loss: 0.485, Accuracy: 0.781
Epoch 16/ 200, Loss: 0.511, Accuracy: 0.750
Epoch 17/ 200, Loss: 0.735, Accuracy: 0.625
Epoch 18/ 200, Loss: 0.653, Accuracy: 0.656
Epoch 19/ 200, Loss: 0.296, Accuracy: 0.844
Epoch 20/ 200, Loss: 0.374, Accuracy: 0.844
Epoch 21/ 200, Loss: 0.434, Accuracy: 0.812
Epoch 22/ 200, Loss: 0.413, Accuracy: 0.781
Epoch 23/ 200, Loss: 0.504, Accuracy: 0.7

Epoch 189/ 200, Loss: 0.375, Accuracy: 0.812
Epoch 190/ 200, Loss: 0.368, Accuracy: 0.844
Epoch 191/ 200, Loss: 0.273, Accuracy: 0.906
Epoch 192/ 200, Loss: 0.297, Accuracy: 0.844
Epoch 193/ 200, Loss: 0.337, Accuracy: 0.812
Epoch 194/ 200, Loss: 0.293, Accuracy: 0.875
Epoch 195/ 200, Loss: 0.203, Accuracy: 0.938
Epoch 196/ 200, Loss: 0.329, Accuracy: 0.844
Epoch 197/ 200, Loss: 0.303, Accuracy: 0.906
Epoch 198/ 200, Loss: 0.397, Accuracy: 0.844
Epoch 199/ 200, Loss: 0.504, Accuracy: 0.719
Epoch 200/ 200, Loss: 0.473, Accuracy: 0.781
